In [2]:
# Goal

# Group customers by first purchase time
# Track how long they stay active
# Understand retention vs churn
# Dataset: Retail Transactions

# Core Idea
# A cohort = customers who started at the same time.
# Example:
# Jan cohort → customers whose first purchase was in January
# Feb cohort → first purchase in February
# Then we track:
# Do they buy again in later months?

In [4]:
# Step 1: Prepare Dates
import pandas as pd

df = pd.read_csv('Datasets/retail_sales_dataset.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['order_month'] = df['Date'].dt.to_period('M')

In [6]:
df

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,order_month
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,2023-11
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,2023-02
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,2023-01
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,2023-05
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,2023-05
...,...,...,...,...,...,...,...,...,...,...
995,996,2023-05-16,CUST996,Male,62,Clothing,1,50,50,2023-05
996,997,2023-11-17,CUST997,Male,52,Beauty,3,30,90,2023-11
997,998,2023-10-29,CUST998,Female,23,Beauty,4,25,100,2023-10
998,999,2023-12-05,CUST999,Female,36,Electronics,3,50,150,2023-12


In [10]:
# Step 2: Find Each Customer’s First Month
first_purchase = (
    df.groupby('Customer ID')['order_month']
      .min()
      .reset_index(name='cohort_month')
)

df = df.merge(first_purchase, on='Customer ID', how='left')

In [12]:
# Step 3: Calculate Cohort Index (Month Number)
df['cohort_index'] = (
    df['order_month'] - df['cohort_month']
).apply(lambda x: x.n)


# Meaning:
# 0 → first month
# 1 → next month
# 2 → month after that

In [14]:
# Step 4: Build Cohort Table (Customer Count)
cohort_table = (
    df.groupby(['cohort_month', 'cohort_index'])['Customer ID']
      .nunique()
      .reset_index()
)


In [16]:
cohort_table

,cohort_month,cohort_index,Customer ID
0,2023-01,0,76
1,2023-02,0,85
2,2023-03,0,73
3,2023-04,0,86
4,2023-05,0,105
5,2023-06,0,77
6,2023-07,0,72
7,2023-08,0,94
8,2023-09,0,65
9,2023-10,0,96


In [18]:
# Step 5: Convert to Retention Matrix
cohort_pivot = cohort_table.pivot(
    index='cohort_month',
    columns='cohort_index',
    values='Customer ID'
)

cohort_size = cohort_pivot.iloc[:, 0]
retention = cohort_pivot.divide(cohort_size, axis=0)

retention

cohort_index,0
cohort_month,
2023-01,1.0
2023-02,1.0
2023-03,1.0
2023-04,1.0
2023-05,1.0
2023-06,1.0
2023-07,1.0
2023-08,1.0
2023-09,1.0
